In [29]:
import numpy as np
import pandas as pd
import sportsref
import os, platform
import glob
import re
from sportsref import nfl
from pyquery import PyQuery as pq

# SCRAPING

In [14]:
# set path based on if im at work/home laptop or cpu
if platform.system()=='Darwin':
    path = '/Users/phil/Google Drive/projects/nfl/data'
elif os.environ.get("USERNAME")=='phil':
    path = 'C:/Users/phil/Google Drive/nfl/data'
elif os.environ.get("USERNAME")=='lyncp010':
    path = 'C:/Users/lyncp010/projects/nfl/data'
print path

C:/Users/lyncp010/projects/nfl/data


## boxscore pages

In [20]:
# scraping parameters
# season start/end
ss = 2016
se = 2016

# week start/end
ws = 17
we = 17

In [21]:
# boxscore dfs
gameInfo = pd.DataFrame()
starters = pd.DataFrame()
snapCounts = pd.DataFrame()
statsTeam = pd.DataFrame()
statsOffense = pd.DataFrame()
statsDefense = pd.DataFrame()
statsReturns = pd.DataFrame()
statsKicking = pd.DataFrame()
passDirections = pd.DataFrame()

# playByPlay = pd.DataFrame()

In [22]:
## append to boxscore dfs
print 'Scraping...'
for s in range(ss, se+1):
    season_boxscores = nfl.boxscores.get_season_boxscores_IDs(s)
    bsIDs = season_boxscores.loc[ws: we]
    n_games = len(bsIDs)
    for i, bsID in enumerate(bsIDs):
        print '\rSeason {} - Game {} ({}/{})'.format(s, bsID, i+1, n_games),
        bs = nfl.boxscores.BoxScore(bsID)
        gameInfo = pd.concat([gameInfo, bs.game_info()], ignore_index=True)
        starters = pd.concat([starters, bs.starters()], ignore_index=True)
        snapCounts = pd.concat([snapCounts, bs.snap_counts()], ignore_index=True)
        statsTeam = pd.concat([statsTeam, bs.stats_team()], ignore_index=True)
        statsOffense = pd.concat([statsOffense, bs.stats_offense()], ignore_index=True)
        statsDefense = pd.concat([statsDefense, bs.stats_defense()], ignore_index=True)
        statsReturns = pd.concat([statsReturns, bs.stats_returns()], ignore_index=True)
        statsKicking = pd.concat([statsKicking, bs.stats_kicking()], ignore_index=True)
        passDirections = pd.concat([passDirections, bs.pass_directions()], ignore_index=True)
        # try:
        #     playByPlay = pd.concat([playByPlay, bs.pbp()], ignore_index=True)
        #     #playByPlay.to_csv(path + '/all/playByPlay_s{:02d}w{:02d}_s{:02d}w{:02d}.csv'.format(ss%100, ws, se%100, we), index=False)
        # except:
        #     print 'BAD', bsID
        #     pass

# changing season/week to int
for df in [gameInfo, passDirections, snapCounts, starters, statsDefense, statsKicking, statsOffense, statsReturns, statsTeam]:
    if 'season' in df: df['season'] = df['season'].astype(int)
    if 'week' in df: df['week'] = df['week'].astype(int)

print '\nDone!'

Scraping...
Season 2016 - Game 201701010det (16/16) 
Done!


In [24]:
# output to csv
gameInfo.to_csv(path + '/current_season/gameInfo_s{:02d}w{:02d}_s{:02d}w{:02d}.csv'.format(ss%100, ws, se%100, we), index=False)
starters.to_csv(path + '/current_season/starters_s{:02d}w{:02d}_s{:02d}w{:02d}.csv'.format(ss%100, ws, se%100, we), index=False)
snapCounts.to_csv(path + '/current_season/snapCounts_s{:02d}w{:02d}_s{:02d}w{:02d}.csv'.format(ss%100, ws, se%100, we), index=False)
statsTeam.to_csv(path + '/current_season/statsTeam_s{:02d}w{:02d}_s{:02d}w{:02d}.csv'.format(ss%100, ws, se%100, we), index=False)
statsOffense.to_csv(path + '/current_season/statsOffense_s{:02d}w{:02d}_s{:02d}w{:02d}.csv'.format(ss%100, ws, se%100, we), index=False)
statsDefense.to_csv(path + '/current_season/statsDefense_s{:02d}w{:02d}_s{:02d}w{:02d}.csv'.format(ss%100, ws, se%100, we), index=False)
statsReturns.to_csv(path + '/current_season/statsReturns_s{:02d}w{:02d}_s{:02d}w{:02d}.csv'.format(ss%100, ws, se%100, we), index=False)
statsKicking.to_csv(path + '/current_season/statsKicking_s{:02d}w{:02d}_s{:02d}w{:02d}.csv'.format(ss%100, ws, se%100, we), index=False)
passDirections.to_csv(path + '/current_season/passDirections_s{:02d}w{:02d}_s{:02d}w{:02d}.csv'.format(ss%100, ws, se%100, we), index=False)

# playByPlay.to_csv(path + '/current_season/playByPlay_s{:02d}w{:02d}_s{:02d}w{:02d}.csv'.format(ss%100, ws, se%100, we), index=False)

## team pages

In [25]:
# scraping parameters
# season start/end
ss = 2016
se = 2016

# week start/end
ws = 1
we = 21

In [26]:
# team dfs
injuryStatus = pd.DataFrame()
teamRoster = pd.DataFrame()

In [27]:
# append to team dfs
print 'Scraping...'
for s in range(ss, se+1):
    for i, team in enumerate(nfl.teams.list_teams(s)):
        print '\rSeason {} - Team {} ({}/32)'.format(s, team, i+1),
        injuryStatus = pd.concat([injuryStatus, nfl.teams.Team(team).injury_status(s)], ignore_index=True)
        teamRoster = pd.concat([teamRoster, nfl.teams.Team(team).roster(s)], ignore_index=True)

# changing season/week to int
for df in [injuryStatus, teamRoster]:
    if 'season' in df: df['season'] = df['season'].astype(int)
    if 'week' in df: df['week'] = df['week'].astype(int)

print '\nDone!'

Scraping...
Season 2016 - Team chi (32/32) 
Done!


In [31]:
# remove old csvs
dfs = ['injuryStatus', 'teamRoster']
files = []
print 'Deleting...'
for df in dfs:
    f = glob.glob(os.path.join(path, 'current_season', "{}*.csv".format(df)))
    print f
    os.remove(f[0])

['C:/Users/lyncp010/projects/nfl/data\\current_season\\injuryStatus_s16w01_s16w16.csv']
['C:/Users/lyncp010/projects/nfl/data\\current_season\\teamRoster_s16w01_s16w16.csv']


In [32]:
# output to csv
injuryStatus.to_csv(path + '/current_season/injuryStatus_s{:02d}w01_s{:02d}w{:02d}.csv'.format(ss%100, se%100, we), encoding='utf-8', index=False)
teamRoster.to_csv(path + '/current_season/teamRoster_s{:02d}w01_s{:02d}w{:02d}.csv'.format(ss%100, se%100, we), encoding='utf-8', index=False)

## combining csvs

In [33]:
# list of csv tables
dfs = ['gameInfo'
      ,'passDirections'
      ,'snapCounts'
      ,'starters'
      ,'statsDefense'
      ,'statsKicking'
      ,'statsOffense'
      ,'statsReturns'
      ,'statsTeam'
      ,'injuryStatus'
      ,'teamRoster'
# dfs = ['playByPlay'
]

In [34]:
# combine all csvs into df
d = {}
m = re.compile(r's\d{2}w\d{2}')
for df in dfs:
    files = []
    files += glob.glob(os.path.join(path, 'previous_season', "{}*.csv".format(df)))
    files += glob.glob(os.path.join(path, 'current_season', "{}*.csv".format(df)))
    data_range = '{}_{}'.format(m.findall(files[0])[0], m.findall(files[-1])[1])
    print '{} - {}'.format(df, data_range)
    for f in files: print f
    d[df] = pd.concat((pd.read_csv(f, low_memory=False) for f in files), ignore_index=True)

gameInfo - s02w01_s16w21
C:/Users/lyncp010/projects/nfl/data\previous_season\gameInfo_s02w01_s15w21.csv
C:/Users/lyncp010/projects/nfl/data\current_season\gameInfo_s16w01_s16w16.csv
C:/Users/lyncp010/projects/nfl/data\current_season\gameInfo_s16w17_s16w17.csv
C:/Users/lyncp010/projects/nfl/data\current_season\gameInfo_s16w18_s16w21.csv
passDirections - s02w01_s16w21
C:/Users/lyncp010/projects/nfl/data\previous_season\passDirections_s02w01_s15w21.csv
C:/Users/lyncp010/projects/nfl/data\current_season\passDirections_s16w01_s16w16.csv
C:/Users/lyncp010/projects/nfl/data\current_season\passDirections_s16w17_s16w17.csv
C:/Users/lyncp010/projects/nfl/data\current_season\passDirections_s16w18_s16w21.csv
snapCounts - s02w01_s16w21
C:/Users/lyncp010/projects/nfl/data\previous_season\snapCounts_s02w01_s15w21.csv
C:/Users/lyncp010/projects/nfl/data\current_season\snapCounts_s16w01_s16w16.csv
C:/Users/lyncp010/projects/nfl/data\current_season\snapCounts_s16w17_s16w17.csv
C:/Users/lyncp010/projects

In [35]:
# export combined dfs
for df in dfs:
    d[df].to_csv(path + '/all/{}_{}.csv'.format(df, data_range), encoding='utf-8', index=False)

## coaches

In [ ]:
coaches_df = pd.DataFrame()

In [ ]:
# scrape coach id for each season / team
for s in range(2002, 2016+1):
    for i, tm_nm in enumerate(nfl.teams.list_teams(s)):
        print '\rSeason {} - Team {}'.format(s, i+1),
        tm = nfl.Team(tm_nm)
        try:
            coach_array = tm.head_coaches_by_game(s)
            coach_df = pd.DataFrame(data = coach_array,
                                    columns = ['coach'],
                                    index=[[2016]*len(coach_array), [tm_nm]*len(coach_array), range(1, len(coach_array)+1)]
                                   )
            coach_df.index.names = ['seaon', 'team', 'week']
            coaches_df = pd.concat([coaches_df, coach_df])
        except:
            print 'BAD {} - {}'.format(s, tm_nm)
            pass

In [ ]:
from pyquery import PyQuery as pq

In [ ]:
coach_names = {}

In [ ]:
# get coache names from their coach id
all_coaches = coaches_df['coach'].unique()
for i, c in enumerate(all_coaches):
    print '\rCoach {} / {}'.format(i+1, len(all_coaches)),
    doc = pq('http://www.pro-football-reference.com/coaches/{}.htm'.format(c))
    try:
        coach_names[c] = doc('div#meta h1:first').text()
    except:
        print 'BAD {}'.format(c)
        coach_names[c] = ''

In [ ]:
coaches_df['coach_name'] = coaches_df['coach'].map(coach_names)

In [ ]:
coaches_df = coaches_df.rename(columns={'coach':'coach_id'})

In [ ]:
coaches_df.to_csv(path + '/all/coaches_s2002w01_s2016w21.csv')#, index=False)

# testing

#### fixing stuff

#### pulling future (ff playoff) game infos

In [ ]:
season = 2016

In [ ]:
nextGames = getSeasonBoxScores(season)

# get index of first game of week 14
print nextGames.tolist().index('201612080kan')

In [ ]:
url = sportsref.nfl.BASE_URL + '/years/{}/games.htm'.format(season)
doc = pq(sportsref.utils.getHTML(url))
table = doc('table#games')
playoffGames = sportsref.utils.parseTable(table).loc[192: 192 + 16*3 - 1]

# adding/fixing cols
playoffGames['season'] = season
playoffGames['week'] = playoffGames['week_num'].astype(int)
playoffGames['bsID'] = playoffGames['boxscore_word']
playoffGames['team'] = playoffGames['loser']
playoffGames['opponent'] = playoffGames['winner']
playoffGames['home'] = 1

In [ ]:
# copy df with cols needed
df = playoffGames[['season', 'week', 'bsID', 'team', 'opponent', 'home']].copy()

# for each game, duplicate row and swap home/away teams, home indicator
for i,r in df.iterrows():
    foo = df.loc[i].copy()
    foo['team'], foo['opponent'], foo['home'] = foo['opponent'], foo['team'], int(not(df.loc[i]['home']))
    df = df.append(foo)

# sort df
df = (df.sort_values(by=['season', 'week', 'bsID', 'home'])
        .reset_index()
        .drop(['index'], axis=1))

#### team pages messed up